##**Notebook PC#11**

## Encoder-Decoder LSTM for Natural Language Processing.

**Professor:** Fernando J. Von Zuben <br>
**Aluno(a):** Gabriel Toffanetto França da Rocha - 289320<br>
**Aluno(a):** Maria Fernanda Paulino Gomes - 206745<br>

In [ ]:
from random import seed
from random import randint
from numpy import array
from numpy import argmax

In [ ]:
def random_sum_pairs(n_examples, n_numbers, largest):
    X,y = list(), list()
    for i in range(n_examples):
        in_pattern=[randint(1,largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X,y

In [ ]:
seed(1)
n_samples =1
n_numbers = 2
largest = 10
X,y = random_sum_pairs(n_samples, n_numbers, largest)
print(X,y)

[[3, 10]] [13]


In [ ]:
from math import ceil
from math import log10

In [ ]:
def to_string(X,y,n_numbers,largest):
    max_length = n_numbers*ceil(log10(largest+1)) + n_numbers - 1
    Xstr = list()
    for pattern in X:
        strp = '+'.join([str(n) for n in pattern])
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        Xstr.append(strp)
    maxlength = ceil(log10(n_numbers*(largest+1)))
    ystr = list()
    for pattern in y:
        strp = str(pattern)
        strp = ''.join([' 'for _ in range(maxlength-len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr

In [ ]:
seed(1)
n_samples = 1
n_numbers = 2
largest = 10

In [ ]:
X,y = random_sum_pairs(n_samples, n_numbers, largest)
print(X,y)

X,y = to_string(X,y,n_numbers,largest)
print(X,y)

[[3, 10]] [13]
[' 3+10'] ['13']


In [ ]:
alphabet = ['0','1','2','3','4','5','6','7','8','9','+',' ']

In [ ]:
def integer_encode(X,y,alphabet):
    char_to_int = dict((c,i) for i,c in enumerate(alphabet))
    Xenc = list()
    for pattern in X:
        integer_encoded = [char_to_int[char] for char in pattern]
        Xenc.append(integer_encoded)
    yenc = list()
    for pattern in y:
        integer_encoded = [char_to_int[char] for char in pattern]
        yenc.append(integer_encoded)
    return Xenc, yenc

In [ ]:
X,y = integer_encode(X,y,alphabet)

In [ ]:
print(X,y)

[[11, 3, 10, 1, 0]] [[1, 3]]


In [ ]:
def one_hot_encode(X,y,max_int):
    Xenc = list()
    for seq in X:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc.append(pattern)

    yenc = list()
    for seq in y:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc.append(pattern)
    return Xenc, yenc

In [ ]:
X,y = one_hot_encode(X,y,len(alphabet))

In [ ]:
print(X,y)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]] [[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]]


In [ ]:
def generate_data(n_samples,n_numbers, largest, alphabet):
    X,y = random_sum_pairs(n_samples,n_numbers,largest)
    X,y = to_string(X,y,n_numbers,largest)
    X,y = integer_encode(X,y,alphabet)
    X,y = one_hot_encode(X,y,len(alphabet))
    X,y = array(X), array(y)
    return X,y

In [ ]:
def invert(seq,alphabet):
    int_to_char = dict((i,c) for i,c in enumerate(alphabet))
    strings  = list()
    for pattern in seq:
        string = int_to_char[argmax(pattern)]
        strings.append(string)
    return ''.join(strings)

In [ ]:
n_terms = 3
largest = 10
alphabet = [str(x) for x in range(10)] + ['+', ' ']

In [ ]:
n_chars = len(alphabet)
n_in_seq_length = n_terms*ceil(log10(largest+1)) +n_terms-1
n_out_seq_length = ceil(log10(n_terms*(largest+1)))

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Dense

In [ ]:
model = Sequential()
model.add(LSTM(75, input_shape=(n_in_seq_length,n_chars)))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(50,return_sequences=True))
model.add(TimeDistributed(Dense(n_chars,activation='softmax')))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 75)                26400     
                                                                 
 repeat_vector (RepeatVecto  (None, 2, 75)             0         
 r)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 2, 50)             25200     
                                                                 
 time_distributed (TimeDist  (None, 2, 12)             612       
 ributed)                                                        
                                                                 
Total params: 52212 (203.95 KB)
Trainable params: 52212 (203.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
X,y = generate_data(75000,n_terms,largest,alphabet)
model.fit(X,y,epochs=1,batch_size=10)

7500/7500 [==============================] - 61s 8ms/step - loss: 0.4006 - accuracy: 0.8773


In [ ]:
X,y = generate_data(100,n_terms,largest,alphabet)
loss,acc = model.evaluate(X,y,verbose=0)
print('Loss: %f, Accuracy: %f' %(loss,acc*100))

Loss: 0.029399, Accuracy: 99.500000


In [ ]:
for _ in range(10):
    X,y = generate_data(1,n_terms,largest,alphabet)
    yhat = model.predict(X,verbose=0)
    in_seq = invert(X[0],alphabet)
    out_seq = invert(y[0],alphabet)
    predicted = invert(yhat[0],alphabet)
    print('%s = %s (expect %s)' %(in_seq,predicted,out_seq))

   3+7+5 = 15 (expect 15)
  2+10+6 = 18 (expect 18)
   4+5+9 = 18 (expect 18)
   2+2+4 =  8 (expect  8)
 10+7+10 = 27 (expect 27)
  10+2+3 = 15 (expect 15)
   8+4+1 = 13 (expect 13)
   8+1+3 = 12 (expect 12)
   5+9+6 = 20 (expect 20)
   1+5+8 = 14 (expect 14)


<font color="green">
Atividade (a) <br>
Como são gerados os dados de treinamento?
</font>

**Resposta:** A função *generate_data* gera os dados de treinamento, realizando uma série de transformações nos dados.

A função é dada pelo seguinte código:

```
def generate_data(n_samples,n_numbers, largest, alphabet):
    X,y = random_sum_pairs(n_samples,n_numbers,largest)
    X,y = to_string(X,y,n_numbers,largest)
    X,y = integer_encode(X,y,alphabet)
    X,y = one_hot_encode(X,y,len(alphabet))
    X,y = array(X), array(y)
    return X,y
```

Tem-se que a função *random_sum_pairs* gera pares de números aleatórios com base nos parâmetros de entrada: *n_examples* (número de exemplos de pares entrada-saída a serem gerados), *n_numbers* (número de inteiros em cada lista de entrada), *largest* (maior valor que cada inteiro de entrada X pode assumir).

A função *to_string* converte os números gerados para *strings*. A função *integer_encode* utiliza um alfabeto de símbolos para codificar as *strings* de entrada (x) e a saída esperada (y), essa função converte cada símbolo de acordo com a posição de índices da lista *alphabet*.

A função *one_hot_encode* converte a saída numérica para uma representação binária única (*one-hot encoding*), onde cada símbolo é visto como uma classe separada, evitando falsas interpretações pela rede neural.

Por fim, a última linha da função *generate_data* garante que tanto x quanto y estejam em formato de *array* antes de serem retornados.


Dessa forma, os dados de treinamento são gerados, a partir da geração de pares de números e duas somas esperadas, dessa forma, combinando o alfabeto e gerando operações obtém-se "frases" para que o modelo aprenda a manipular esses símbolos durante treinamento, realizando corretamente as operações matemáticas de forma semântica, e não algébrica. Durante o processo, são realizadas conversões intermediárias (como *integer_encode*) para preparar os dados no formato *one-hot encoding* que é o formato mais adequado de representar o alfabeto para o aprendizado de redes neurais.



<font color="green">
Atividade (b) <br>
Como uma calculadora simples pode operar baseada no conceito de tradução de frases, ou seja, sem realizar operações algébricas?
</font>

**Resposta:** Uma calculadora simples, baseada no conceito de tradução de frases opera transformando expressões matemáticas em suas respectivas soluluções sem realizar operações algébricas diretamente. O modelo de tradução de sequência para sequência (*seq2seq*) é usado para mapear uma expressão matemática.

O modelo trata a expressão de entrada como uma sequência de símbolos e aprende a prever a sequência correta de saída. Este processo pode ser dividido da seguinte forma:

*   Codificação de Entrada: a expressão matemática a ser mapeada é *tokenizada* e convertida em uma sequência de vetores *one-hot encoded*. Uma camada LSTM processa essa sequência codificando a informação em um vetor de comprimento fixo que captura o contexto da expressão;
*   Vetor de Contexto: o vetor de contexto gerado pela LSTM, captura as informações essenciais da sequência de entrada;
*   Decodificação para a Saída: uma camada *ReapeatVector* repeto o vetor de contexto para que ele corresponda ao comprimento da sequência de saída esperada. Uma segunda camada LSTM decodifica esse vetor repetido para gerar a sequência de saída passo a passo. Cada símbolo da sequência de saída é gerado um por vez, até que a expressão completa seja traduzida.
*   Camada *Time Distributed*: aplicada uma camada densa a cada elemento da sequência de saída para produzir a saída final, transformando os vetores intermediários em símbolos compreensíveis.
*  Função *Softmax* e Função de Perda: Entropia Cruzada: a camada *softmax* transforma a saída da rede em probabilidades, já a função de perda mede a diferença entre a distribuição de probabilidade prevista e a distribuição verdadeira (a saída correta codificada como *one-hot*).


Durante o processo de treinamento, o modelo aprende a reconhecer padrões nas sequências de entrada e saída. Cada iteração ajusta os pesos da rede para minimizar a perda da entropia cruzada. A camada *RepeatVector* assegura que o vetor de contexto seja adequadamente utilizado pelo decodificador. Com o tempo, o modelo tende a melhorar a capacidade de traduzir expressões matemáticas em suas respectivas soluções, tornando-se então, uma calculadora eficiente.

Em suma, o modelo não realiza operações algébricas diretamente, mas mapeia espressões matemáticas para suas soluções através de um processo de aprendizado baseado em *seq2seq*. Ele transforma a entrada em uma representação vetorial, utilizando LSTMs para codificar e decodificar essa representação, e aplica técnicas de *machine learning* para ajustar melhor suas previsões.

Pode-se correlacionar tal solução para a calculadora com a Sala Chinesa de Searle. Com as entradas em texto, onde se tem os números e sinais matemáticos como caracteres de um alfabeto, a rede neural aprende a combiná-los de acordo com o contexto para entregrar o resultado correto da operação matemática, porém, sem resolver a expressão propriamente dita, ou seja, a calculadora não realiza as operações algébricas, mas age como estivesse.









